In [55]:
import numpy as np 
import cv2
import dlib
from google.colab.patches import cv2_imshow
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import io
from PIL import Image

In [ ]:
VIDEO_HTML = """
<video autoplay
 width=%d height=%d style='cursor: pointer;'></video>
<script>

var video = document.querySelector('video')

navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)
  
var data = new Promise(resolve=>{
  video.onclick = ()=>{
    var canvas = document.createElement('canvas')
    var [w,h] =[video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    video.srcObject.getVideoTracks()[0].stop()
    video.replaceWith(canvas)
    resolve(canvas.toDataURL('image/jpeg', %f))
  }
})
</script>
"""
def take_photo(filename='photo.jpg', quality=0.8, size=(800,600)):
  print("click on image to take a photo")
  display(HTML(VIDEO_HTML % (size[0],size[1],quality)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  f = io.BytesIO(binary)
  return np.asarray(Image.open(f))

img = take_photo()
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [160]:
def add_hat(img_or, hat_img):

    img = np.copy(img_or)

    r,g,b,a = cv2.split(hat_img) 
    rgb_hat = cv2.merge((r,g,b))

    predictor_path = "shape_predictor_5_face_landmarks.dat"
    predictor = dlib.shape_predictor(predictor_path)  

    detector = dlib.get_frontal_face_detector()

    dets = detector(img, 1)

    if len(dets)>0:  
        for d in dets:
            x, y, w, h = d.left(),d.top(), d.right()-d.left(), d.bottom()-d.top()
            shape = predictor(img, d)
            point1 = shape.part(0)
            point2 = shape.part(2)
            eyes_center = ((point1.x+point2.x)/2, (point1.y+point2.y)/2)
            scaling_factor = (w / rgb_hat.shape[1]) * 1.5
            resized_hat_w = int(rgb_hat.shape[1] * scaling_factor)
            resized_hat_h = int(rgb_hat.shape[0] * scaling_factor)

            resized_hat = cv2.resize(rgb_hat,(resized_hat_w,resized_hat_h))
            if resized_hat_h > y:
                resized_hat_h = y-1
            mask = cv2.resize(a, (resized_hat_w,resized_hat_h))
            mask_inv =  cv2.bitwise_not(mask)
            start_y = y - resized_hat_h
            end_y = y
            start_x = round(eyes_center[0] - resized_hat_w/2)
            end_x = round(eyes_center[0] + resized_hat_w/2)
            bg_roi = img[start_y: end_y, start_x: end_x]
            bg_roi = bg_roi.astype(float)
            mask_inv = cv2.merge((mask_inv, mask_inv, mask_inv))
            alpha = mask_inv.astype(float)/255
            alpha = cv2.resize(alpha,(bg_roi.shape[1],bg_roi.shape[0]))
            bg = cv2.multiply(alpha, bg_roi)
            bg = bg.astype('uint8')
            hat = cv2.bitwise_and(resized_hat,resized_hat,mask = mask)
            hat = cv2.resize(hat,(bg_roi.shape[1], bg_roi.shape[0]))
            add_hat = cv2.add(bg,hat)
            
            img[start_y:end_y, start_x: end_x] = add_hat

            return img

In [ ]:
hat_img = cv2.imread("hat_cowboy.png",-1)

output = add_hat(img,hat_img)

cv2_imshow(output)    